In [1]:
import pandas as pd
# for循环显示进度条
from tqdm import tqdm
import warnings
import gc
import os
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
from pylab import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

D:\Anaconda\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df_train = pd.read_excel('train.xlsx')
df_test = pd.read_excel('test_B榜.xlsx')

In [3]:
# 合并训练集测试集
df_feature = df_train.append(df_test, sort=False)

# step1:数据分析

In [4]:
# 数据中 ？ 及缺失值处理
chars = ['CUST_UID','MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO','SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU']
for f in list(df_feature.select_dtypes('object')):
    if f in chars:
        # char类型缺失值填补
        df_feature[f]=df_feature[f].fillna("na")
        # continue
    
    df_feature[f] = df_feature[f].replace('?', 0)
#     # 数值类型缺失值填补
#     df_feature[f]=df_feature[f].fillna(median(df_feature[f].dropna()))

In [5]:
feature_names = list(filter(lambda x: x not in ['LABEL','CUST_UID',
                                                # ---------------------------
                                                'AGN_CNT_RCT_12_MON',
                                                'NB_RCT_3_MON_LGN_TMS_AGV',
                                                'AGN_CUR_YEAR_AMT',
                                                'AGN_AGR_LATEST_AGN_AMT',
                                                'COUNTER_CUR_YEAR_CNT_AMT,'
                                                'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
                                                'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
                                                'MON_12_AGV_ENTR_ACT_CNT',
                                                'MON_12_AGV_LVE_ACT_CNT',
                                                'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT',
                                                'CUR_YEAR_COUNTER_ENCASH_CNT',
                                                'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
                                                'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
                                                'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
                                                # 'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL',   (重要特征)
                                                'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR',
                                                'EMP_NBR',
                                                'REG_CPT',
                                                'HLD_DMS_CCY_ACT_NBR',
                                                'REG_DT',
                                                'OPN_TM',
                                                'HLD_FGN_CCY_ACT_NBR',
                                                # ---------------------
                                                'COUNTER_CUR_YEAR_CNT_AMT',
                                                'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
                                                'MON_12_CUST_CNT_PTY_ID',
                                                'WTHR_OPN_ONL_ICO',
                                                'LGP_HLD_CARD_LVL',
                                                'NB_CTC_HLD_IDV_AIO_CARD_SITU',
                                                # ---------------------
                                                'ICO_CUR_MON_ACM_TRX_TM',
                                                'ICO_CUR_MON_ACM_TRX_AMT',
                                                'MON_12_EXT_SAM_TRSF_IN_AMT',
                                                'MON_12_EXT_SAM_AMT',
                                                'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
                                                'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT',
                                                'MON_12_EXT_SAM_TRSF_OUT_AMT',
                                                'CUR_YEAR_MON_AGV_TRX_CNT',
                                                'MON_12_AGV_TRX_CNT',
                                                'MON_12_ACM_ENTR_ACT_CNT',
                                                'MON_12_ACM_LVE_ACT_CNT',
                                                'MON_6_50_UP_ENTR_ACT_CNT',
                                                'MON_6_50_UP_LVE_ACT_CNT',
                                                'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
                                                'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
                                                'AGN_CUR_YEAR_WAG_AMT',
                                                'CUR_MON_COR_DPS_MON_DAY_AVG_BAL',
                                                'MON_12_TRX_AMT_MAX_AMT_PCTT',
                                                'SHH_BCK',
                                                'AI_STAR_SCO',
                                                'CUR_YEAR_MID_BUS_INC',
                                                'COR_KEY_PROD_HLD_NBR',
                                                'LAST_12_MON_MON_AVG_TRX_AMT_NAV',
                                               ], df_feature.columns))
df_feature[feature_names].corr()
# COUNTER_CUR_YEAR_CNT_AMT, PUB_TO_PRV_TRX_AMT_CUR_YEAR, MON_12_CUST_CNT_PTY_ID, CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL
# WTHR_OPN_ONL_ICO, LGP_HLD_CARD_LVL, NB_CTC_HLD_IDV_AIO_CARD_SITU

,CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL
CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL,1.0


In [6]:
df_feature[df_feature['LABEL'].isnull()][feature_names].corr()

,CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL
CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL,1.0


In [7]:
# 离散特征
# df_feature[df_feature['LABEL'].notnull()].groupby('NB_RCT_3_MON_LGN_TMS_AGV')['LABEL'].mean().plot(kind='bar',figsize=(8,6))

In [8]:
# 绘制续保影响因素柱状图
# 可以查看离散特征中不同属性值的标签分布差异
# df_feature[df_feature['LABEL'].notnull()].groupby('LABEL')['AGN_CNT_RCT_12_MON'].mean().plot(kind='bar',figsize=(8,6))

In [9]:
# plt.hist(df_feature[df_feature['LABEL'].isnull()]['ICO_CUR_MON_ACM_TRX_TM'],10,stacked=True)

## 编码转换

In [10]:
for f in list(df_feature.select_dtypes('object')):
    # ID不进行转换
    if f in ['CUST_UID']:
        continue
    le = LabelEncoder()
    # .fit_transform先编码再标准化
    df_feature[f] = le.fit_transform(df_feature[f].astype('str')).astype('int')

# 划分训练集测试集

In [11]:
# 将合并后的训练集测试集重新拆分回来
df_train = df_feature[df_feature['LABEL'].notnull()]
df_test = df_feature[df_feature['LABEL'].isnull()]

# step3:模型训练

# LGB

In [12]:
def LGB_predict(fold_id, seed, X_train,Y_train,X_val,Y_val,X_test):
    print("LGB test")
    model = lgb.LGBMClassifier(
                           num_leaves=27,     # 取值应<=2^(max_depth),超过此值会导致过拟合        
                           max_depth=5,       # 树的最大深度,-1表示不限制树深,推荐数值[3,5,6,7,9,12,15,17,25]
                           learning_rate=0.3,  # 通常0.01，0.001，0.003...
                           n_estimators=10000, # 树的个数,越多训练时间增加
                           subsample=0.5,      # 默认为1等价于bagging_fraction，控制每棵树随机采样比例，
                           feature_fraction=0.5,   # 等价于colsample_bytree，每次迭代中随机选择80%的特征来建树
                           min_data_in_leaf = 121, # 每个叶节点最少样本数量，过拟合时候使用
                           reg_alpha = 11,
                           reg_lambda = 2.3,
                           random_state=seed,
                           n_jobs = 2,          # 核心数目,几核CPU
                           metric=None
    )
    print('\nFold_{} Training ================================\n'.format(fold_id+1))
    lgb_model = model.fit(X_train,
                      Y_train,
                      eval_names=['valid_train','test'],
                      eval_set=[(X_train,Y_train),(X_val, Y_val)],
                      verbose=100,              # 隔100代显示一次信息
                      eval_metric='auc',        # 误差函数
                      early_stopping_rounds=10  # 在50轮内验证集指标不提升就停止迭代,经验法则为num_iterations的10%。
                      )
    print('\n训练结束！！！')
    pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    pred_test = lgb_model.predict_proba(X_test, num_iteration=lgb_model.best_iteration_)[:, 1]

    return lgb_model,pred_val,pred_test

# XGB

In [13]:
def XGB_predict(fold_id, seed, X_train,Y_train,X_val,Y_val,X_test):
    print("XGB test")
    # create dataset for lightgbm
    xgb_val = xgb.DMatrix(X_val, label=Y_val)
    xgb_train = xgb.DMatrix(X_train, label=Y_train)
    xgb_test = xgb.DMatrix(X_test)
    # 超参数
    params = {
              'booster': 'gbtree',
              'objective': 'binary:logistic', # 'multi:softmax',多分类的问题; 'multi:softprob', 多分类概率
              'eval_metric': 'auc',
              # 'num_class': 9, # 类别数，与 multisoftmax 并用
              'gamma': 0.1, # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
              'max_depth': 5, # 构建树的深度，越大越容易过拟合
              'alpha': 0, # L1正则化系数
              'lambda': 10, # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
              'subsample': 0.5, # 随机采样训练样本
              'colsample_bytree': 0.5, # 生成树时进行的列采样
              'min_child_weight': 3,
              # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
              # ，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
              # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
              'silent': 0, # 设置成1则没有运行信息输出，最好是设置为0.
              'eta': 0.3, # 如同学习率
              'seed': seed,
              'nthread': -1, # cpu 线程数
              'missing': 1,
              }

    plst = list(params.items())
    watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]
    # 训练模型并保存
    # early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
    print('\nFold_{} Training ================================\n'.format(fold_id+1))
    xgb_model = xgb.train(
                    plst, 
                    xgb_train, 
                    num_boost_round = 5000, 
                    evals=watchlist, 
                    verbose_eval=100,
                    early_stopping_rounds=10
                    )
    print('\n训练结束！！！')

    pred_val = xgb_model.predict(xgb_val, ntree_limit=xgb_model.best_ntree_limit)      # 预测验证集
    pred_test = xgb_model.predict(xgb_test, ntree_limit=xgb_model.best_ntree_limit)     # 预测测试集
    
    return xgb_model,pred_val,pred_test

In [14]:
best_seed = []
best_score = 0
# 指定标签
ycol = 'LABEL'
# for f1 in list(filter(lambda x: x not in [ycol,'CUST_UID'], df_train.columns)):
#     for f2 in list(filter(lambda x: x not in [ycol,'CUST_UID',f1], df_train.columns)):
# for seed in range(2023,5000):
# 定义预测结果表格
prediction = df_test[['CUST_UID']]
prediction['LABEL'] = 0
# AGN_CNT_RCT_12_MON , NB_RCT_3_MON_LGN_TMS_AGV, AGN_CUR_YEAR_AMT, AGN_AGR_LATEST_AGN_AMT, COUNTER_CUR_YEAR_CNT_AMT
# PUB_TO_PRV_TRX_AMT_CUR_YEAR, MON_12_EXT_SAM_NM_TRSF_OUT_CNT, MON_12_AGV_ENTR_ACT_CNT, MON_12_AGV_LVE_ACT_CNT,
# CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT, CUR_YEAR_COUNTER_ENCASH_CNT, MON_12_ACT_OUT_50_UP_CNT_PTY_QTY, MON_12_ACT_IN_50_UP_CNT_PTY_QTY
# LAST_12_MON_COR_DPS_DAY_AVG_BAL, CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL, CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR, 
# EMP_NBR, REG_CPT, HLD_DMS_CCY_ACT_NBR, REG_DT, OPN_TM, HLD_FGN_CCY_ACT_NBR

# COUNTER_CUR_YEAR_CNT_AMT, PUB_TO_PRV_TRX_AMT_CUR_YEAR, MON_12_CUST_CNT_PTY_ID
# WTHR_OPN_ONL_ICO, LGP_HLD_CARD_LVL, NB_CTC_HLD_IDV_AIO_CARD_SITU
feature_names = list(filter(lambda x: x not in ['LABEL','CUST_UID',
                                                # ---------------------------
                                                'AGN_CNT_RCT_12_MON',
                                                'NB_RCT_3_MON_LGN_TMS_AGV',
                                                'AGN_CUR_YEAR_AMT',
                                                'AGN_AGR_LATEST_AGN_AMT',
                                                'COUNTER_CUR_YEAR_CNT_AMT,'
                                                'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
                                                'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
                                                'MON_12_AGV_ENTR_ACT_CNT',
                                                'MON_12_AGV_LVE_ACT_CNT',
                                                'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT',
                                                'CUR_YEAR_COUNTER_ENCASH_CNT',
                                                'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
                                                'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
                                                'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
                                                # 'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL',   (重要特征)
                                                'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR',
                                                'EMP_NBR',
                                                'REG_CPT',
                                                'HLD_DMS_CCY_ACT_NBR',
                                                'REG_DT',
                                                'OPN_TM',
                                                'HLD_FGN_CCY_ACT_NBR',
                                                # ---------------------
                                                'COUNTER_CUR_YEAR_CNT_AMT',
                                                'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
                                                'MON_12_CUST_CNT_PTY_ID',
                                                'WTHR_OPN_ONL_ICO',
                                                'LGP_HLD_CARD_LVL',
                                                'NB_CTC_HLD_IDV_AIO_CARD_SITU',
                                                # ---------------------
                                                'ICO_CUR_MON_ACM_TRX_TM',
                                                'ICO_CUR_MON_ACM_TRX_AMT',
                                                'MON_12_EXT_SAM_TRSF_IN_AMT',
                                                'MON_12_EXT_SAM_AMT',
                                                'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
                                                'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT',
                                                'MON_12_EXT_SAM_TRSF_OUT_AMT',
                                                'CUR_YEAR_MON_AGV_TRX_CNT',
                                                'MON_12_AGV_TRX_CNT',
                                                'MON_12_ACM_ENTR_ACT_CNT',
                                                'MON_12_ACM_LVE_ACT_CNT',
                                                'MON_6_50_UP_ENTR_ACT_CNT',
                                                'MON_6_50_UP_LVE_ACT_CNT',
                                                'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
                                                'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
                                                'AGN_CUR_YEAR_WAG_AMT',
                                                'CUR_MON_COR_DPS_MON_DAY_AVG_BAL',
                                                'MON_12_TRX_AMT_MAX_AMT_PCTT',
                                                'SHH_BCK',
                                                'AI_STAR_SCO',
                                                'CUR_YEAR_MID_BUS_INC',
                                                'COR_KEY_PROD_HLD_NBR',
                                                'LAST_12_MON_MON_AVG_TRX_AMT_NAV',
                                               ], df_feature.columns))

seed = 2022
# 记录训练集预测值
oof = []
# 记录特征重要性
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    # 模型训练
    model_lgb,pred_val_lgb,pred_test_lgb = LGB_predict(fold_id, seed, X_train,Y_train,X_val,Y_val,df_test[feature_names])
    # Lgb 模型特征重要性提取
    # df_importance_lgb = pd.DataFrame({'column': feature_names,'importance': model_lgb.feature_importances_,})

    # mode_xgb,pred_val_xgb,pred_test_xgb = XGB_predict(fold_id, seed, X_train,Y_train,X_val,Y_val,df_test[feature_names])
    # xgb 模型特征重要性提取
    # df_importance_xgb = pd.DataFrame({'column': feature_names,'importance': mode_xgb.get_fscore().values()})

    # df_importance_list.append(df_importance_lgb)  # 记录特征重要性

    # 每一折验证集的数据保存
    df_oof = df_train.iloc[val_idx][['CUST_UID', ycol]].copy()
    # pred_val = (pred_val_lgb + pred_val_xgb)/2  # 模型融合
    pred_val = pred_val_lgb
    df_oof['pred'] = pred_val  # 有3列数据ID、真实值、预测值
    oof.append(df_oof)

    # 预测测试集
    # pred_test = (pred_test_lgb + pred_test_xgb)/2
    pred_test = pred_test_lgb
    prediction['LABEL'] += (pred_test/5)  # 每次预测的1/5加进去,乘以0.8防止出现大于一的情况，或者归一化



    del pred_val, pred_test, X_train, Y_train, X_val, Y_val
    del model_lgb,pred_val_lgb,pred_test_lgb,
    # del mode_xgb,pred_val_xgb,pred_test_xgb
    gc.collect()

# 计算训练集auc分数
# df_oof = pd.concat(oof)
# score = roc_auc_score(df_oof['LABEL'], df_oof['pred'])
#     if best_score < score:
#         best_score = score
#         best_seed.append(seed)
#     print('*************','now_seed:',seed ,best_score,'bestseed', best_seed,'*************')

XGB test

Fold_1 Training ================================

[16:41:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "missing", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.89635	val-auc:0.88543
[21]	train-auc:0.90707	val-auc:0.88941

训练结束！！！
XGB test

Fold_2 Training ================================

[16:41:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "missing", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly

[17]	train-auc:0.90600	val-auc:0.89290

训练结束！！！
XGB test

Fold_1 Training ================================

[16:41:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "missing", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.89253	val-auc:0.89953
[17]	train-auc:0.90339	val-auc:0.90249

训练结束！！！
XGB test

Fold_2 Training ================================

[16:41:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "missing", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter a

In [15]:
# 计算训练集auc分数
df_oof = pd.concat(oof)
score = roc_auc_score(df_oof['LABEL'], df_oof['pred'])
score

0.8966298106666667

In [16]:
# # 显示特征重要性排名表
# df_importance = pd.concat(df_importance_list)
# df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
# df_importance

In [17]:
prediction.head()

,CUST_UID,LABEL
0,f5dcfe9b9e8d496d893cfcba8fcac1d4,0.045934
1,68883d45646d44d8b851476058e4500d,0.034004
2,2b93dfa58f10463a937cd42cc42e54f7,0.314899
3,94fc13e16dac4642ab7544d8889e8579,0.035421
4,6be5af4911964a31a0b5cf115799ace6,0.287311


# step4:保存结果

In [18]:
os.makedirs('sub', exist_ok=True)
prediction.to_csv(f'sub/{score}.csv', index=False)
prediction.to_csv(f'sub/sample_submit.txt',sep=' ', index=False, header=False ,escapechar=' ')